In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, TruncatedSVD
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression

Using TensorFlow backend.
C:\Users\r___e\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\r___e\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\r___e\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\r___e\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
df_L = pd.read_csv('TrainData\Plant1_Location3_No_Nan.csv');
df_L_24 = pd.read_csv('TestData\Test_P1_L3_24.csv');
df_L_48 = pd.read_csv('TestData\Test_P1_L3_48.csv');

# Training

In [3]:
y_train = df_L.loc[:,'cond'];
df_L.drop('cond', axis='columns', inplace=True);
df_L.drop('Year', axis='columns', inplace=True);
df_L.drop('Day', axis='columns', inplace=True);
df_L.drop('Month', axis='columns', inplace=True);
df_L.drop('Hour', axis='columns', inplace=True);
df_L.drop('Minute', axis='columns', inplace=True);
df_L_24.drop('Year', axis='columns', inplace=True);
df_L_24.drop('Day', axis='columns', inplace=True);
df_L_24.drop('Month', axis='columns', inplace=True);
df_L_24.drop('Hour', axis='columns', inplace=True);
df_L_24.drop('Minute', axis='columns', inplace=True);
df_L_48.drop('Year', axis='columns', inplace=True);
df_L_48.drop('Day', axis='columns', inplace=True);
df_L_48.drop('Month', axis='columns', inplace=True);
df_L_48.drop('Hour', axis='columns', inplace=True);
df_L_48.drop('Minute', axis='columns', inplace=True);
x_train = df_L;
x_L24 = df_L_24;
x_L48 = df_L_48;
x_L24 = np.vstack([x_train, x_L24]);
x_L48 = np.vstack([x_train, x_L48]);

## PCA

In [4]:
from sklearn.decomposition import PCA, TruncatedSVD

X_p24 = PCA(n_components=5, random_state=42).fit_transform(x_L24);
X_p48 = PCA(n_components=5, random_state=42).fit_transform(x_L48);

## 분리

In [5]:
x_train24 = X_p24[0:x_train.shape[0],:];
x_train48 = X_p48[0:x_train.shape[0],:];
x_test24 = X_p24[x_train.shape[0]:,:];
x_test48 = X_p48[x_train.shape[0]:,:];

## Predict

In [6]:
from imblearn.combine import SMOTEENN

smt = SMOTEENN(sampling_strategy='auto', random_state=42);
X_sm24, y_sm24 = smt.fit_sample(x_train24, y_train)
X_sm48 ,y_sm48 = smt.fit_sample(x_train48, y_train)

In [7]:
lr_model24 = LogisticRegression(solver = 'saga',max_iter = 10000);
lr_model48 = LogisticRegression(solver = 'saga',max_iter = 10000);
lr_model24.fit(X_sm24,y_sm24.ravel());
lr_model48.fit(X_sm48,y_sm48.ravel());

In [8]:
P_24= lr_model24.predict(x_test24);
PP_24 = lr_model24.predict_proba(x_test24);
P_48= lr_model48.predict(x_test48);
PP_48 = lr_model48.predict_proba(x_test48);
PP_24 = np.delete(PP_24,(0), axis=1)
PP_48 = np.delete(PP_48,(0), axis=1)

# Save

In [9]:
df_T = pd.read_csv('TestData\Time_P1_L3_Data.csv');

df_T.drop('tem_in', axis='columns', inplace=True);
df_T.drop('tem_coil', axis='columns', inplace=True);
df_T.drop('tem_out', axis='columns', inplace=True);
df_T.drop('hum_in', axis='columns', inplace=True);
df_T.drop('hum_out', axis='columns', inplace=True);

In [10]:
df_T.head()

,Year,Month,Day,Hour,Minute
0,2019,4,3,12,30
1,2019,4,5,2,30
2,2019,4,5,8,0
3,2019,4,5,15,30
4,2019,4,6,6,30


In [11]:
df_T['24H'] = P_24;
df_T['24H_P'] = PP_24;
df_T['48H'] = P_48;
df_T['48H_P'] = PP_48;

In [12]:
df_T.to_csv('Result_P1_L3.csv',index = False) 